# Overfitting and Regularization with Weight Decay

## Learning from Data: Homework 6

### Questions 2, 3, 4, 5, 6

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_table("in.dta", sep=" +", header=None, engine='python')
train.columns = ["x1", "x2", "y"]
test = pd.read_table("out.dta", sep=" +", header=None, engine='python')
test.columns = ["x1", "x2", "y"]

In [3]:
for df in [train, test]:
    df['one'] = 1
    df['x1^2'] = df['x1']**2
    df['x2^2'] = df['x2']**2
    df['x1x2'] = df['x1']*df['x2']
    df['|x1-x2|'] = np.abs(df['x1'] - df['x2'])
    df['|x1+x2|'] = np.abs(df['x1'] + df['x2'])
    df.reindex(columns = ['one', 'x1', 'x2', 'x1^2', 'x2^2', 'x1x2', '|x1-x2|', '|x1+x2|'])

In [4]:
train = train.reindex(columns = ['one', 'x1', 'x2', 'x1^2', 'x2^2', 'x1x2', '|x1-x2|', '|x1+x2|', 'y'])
test = test.reindex(columns = ['one', 'x1', 'x2', 'x1^2', 'x2^2', 'x1x2', '|x1-x2|', '|x1+x2|', 'y'])

In [5]:
def lin_regression(Xs, Ys):
    # Dot product of pseudo-inverse of X and Y
    return np.dot(np.linalg.pinv(Xs), Ys)

# One step learning: determine the weights
w = lin_regression(train.ix[:,:-1], train['y'])

In [6]:
# Classification error: Compute outputs with current w
# and compare to real outputs
def class_error(Xs, Ys, w):
    Y_est = np.sign(np.dot(Xs, w))
    return np.mean(Y_est != Ys)

In [7]:
# Classification error on training set
class_error(train.ix[:,:-1], train['y'], w)

0.028571428571428571

In [8]:
# Classification error on test set
class_error(test.ix[:,:-1], test['y'], w)

0.084000000000000005

In [9]:
# Regularized linear regression
def regul_lin_regression(Xs, Ys, k):
    l = 10**k
    A = np.linalg.inv(np.dot(Xs.T, Xs) + np.eye(Xs.shape[1])*l)
    B = np.dot(Xs.T, Ys)
    return np.dot(A,B)

# Weights with lambda = -3
w_neg_k = regul_lin_regression(train.ix[:,:-1], train['y'], -3)

In [10]:
#Classification error on train set
#with lambda = -3
class_error(train.ix[:,:-1], train['y'], w_neg_k)

0.028571428571428571

In [11]:
#Classification error on test set
#with lambda = -3
class_error(test.ix[:,:-1], test['y'], w_neg_k)

0.080000000000000002

In [12]:
# Weights with lambda = 3
w_pos_k = regul_lin_regression(train.ix[:,:-1], train['y'], 3)

In [13]:
#Classification error on train set
#with lambda = 3
class_error(train.ix[:,:-1], train['y'], w_pos_k)

0.37142857142857144

In [14]:
#Classification error on test set
#with lambda = 3
class_error(test.ix[:,:-1], test['y'], w_pos_k)

0.436

In [15]:
# Find minimum E_out
def min_E_out(ks):
    errors = np.zeros(len(ks))
    for i,k in enumerate(ks):
        w_est = regul_lin_regression(train.ix[:, :-1], train['y'], k)
        error = class_error(test.ix[:,:-1], test['y'], w_est)
        errors[i] = error   
    min_k = np.argmin(errors)
    return (errors, errors[min_k], ks[min_k])

In [16]:
# Vary k over [2,1,0,-1,-2]
min_E_out([2,1,0,-1,2])

(array([ 0.228,  0.124,  0.092,  0.056,  0.228]), 0.056000000000000001, -1)

In [17]:
# Vary k over range [-10, 10]
min_E_out(np.arange(-10,11))

(array([ 0.084,  0.084,  0.084,  0.084,  0.084,  0.084,  0.084,  0.08 ,
         0.084,  0.056,  0.092,  0.124,  0.228,  0.436,  0.452,  0.456,
         0.456,  0.456,  0.456,  0.456,  0.456]), 0.056000000000000001, -1)